In [64]:
import pandas as pd
import requests
from IPython.display import JSON
import numpy as np

city_country_code = [['berlin','DE'],['hamburg','DE'],['london','GB'],['manchester','GB'],['barcelona','ES']]
OWM_key = 'cf36567c4cb288fd1cd39a18cbfc1b5b'
list_for_df=[]


def weather_data_df(city_country_code):
 for cc in city_country_code:
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={cc[0]},{cc[1]}&appid={OWM_key}&units=metric"
    response = requests.get(url)
    
    weather_json=response.json()
    
    
    for hour in weather_json['list']:
           weather_dict = {}
           weather_dict['city'] = cc[0]
           weather_dict['temperature'] = hour['main'].get('temp', None)
           weather_dict['wind_speed'] = hour['wind'].get('speed', None)
           weather_dict['forecast_time'] = hour.get('dt_txt', None)
#          weather_dict['outlook'] = hour['list']['main'].get('temp', None)
           weather_dict['temperature_feels_like'] = hour['main'].get('feels_like', None)
           weather_dict['pop'] = hour.get('pop', None)
           list_for_df.append(weather_dict)     
        
 return pd.DataFrame(list_for_df)


In [65]:
weather_data = weather_data_df(city_country_code)
condition=[(weather_data['city'] == 'berlin') , (weather_data['city'] == 'hamburg')  , 
           (weather_data['city'] == 'london')  , (weather_data['city'] == 'manchester') ,
          (weather_data['city'] == 'barcelona') ]
choice=[1,2,3,4,5]
weather_data['city_id'] =np.select(condition,choice,default = '') 
weather_data = weather_data.iloc[:,[6,0,1,2,3,4,5]]
weather_data

,city_id,city,temperature,wind_speed,forecast_time,temperature_feels_like,pop
0,1,berlin,13.67,3.20,2023-03-18 12:00:00,12.50,0.00
1,1,berlin,14.32,1.72,2023-03-18 15:00:00,13.32,0.29
2,1,berlin,12.73,0.65,2023-03-18 18:00:00,11.94,0.22
3,1,berlin,10.36,1.90,2023-03-18 21:00:00,9.78,0.00
4,1,berlin,8.89,1.87,2023-03-19 00:00:00,8.01,0.00
...,...,...,...,...,...,...,...
195,5,barcelona,15.08,3.90,2023-03-22 21:00:00,14.60,0.00
196,5,barcelona,14.54,3.39,2023-03-23 00:00:00,14.03,0.00
197,5,barcelona,14.27,2.77,2023-03-23 03:00:00,13.68,0.00
198,5,barcelona,13.83,2.35,2023-03-23 06:00:00,13.20,0.00


In [66]:
!pip install sqlalchemy 
!pip install pymysql
!pip install mysql-connector-python-rf
import sqlalchemy # install if needed
import pymysql
from sqlalchemy import create_engine
import mysql.connector
import socket

schema="gans"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password="Payment_123" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'   

socket.getaddrinfo('localhost', 8080)
cnx = create_engine(con,echo=False)  
df = pd.read_sql('SELECT * FROM weather_data', con)  

weather_data.to_sql('weather_data', con=cnx, if_exists='append', index=False) 

200